In [1]:
import pandas as pd
import numpy as np
import ast
import warnings
from tqdm import tqdm
import re
import tensorflow as tf
from tensorflow import keras

# 데이터 불러오기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/god/data/'
df = pd.read_csv(path + 'for_vector_table.csv')

In [4]:
df.head()

,id,ex_sentence
0,1,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...
1,1,쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 ...
2,1,논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 ...
3,2,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...
4,2,컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 따라 고소천사벽화마을과 자산마...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731931 entries, 0 to 731930
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           731931 non-null  int64 
 1   ex_sentence  731931 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.2+ MB


In [9]:
df.isna().sum()

id             0
ex_sentence    0
dtype: int64

In [12]:
df['ex_sentence'][0]

'전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.'

# Vector

In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.0 MB/s eta 0:00:00


In [14]:
from transformers import *
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/model.safetensors
Loaded 177,853,440 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a 

# 벡터화 테스트

In [22]:
df2 = df[:5]

In [23]:
df2

,id,ex_sentence
0,1,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...
1,1,쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 ...
2,1,논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 ...
3,2,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...
4,2,컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 따라 고소천사벽화마을과 자산마...


In [ ]:
def get_sentence_vector(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    input_ids = tf.constant([input_ids])
    outputs = model(input_ids)
    sentence_vector = outputs.pooler_output
    return sentence_vector.numpy()

In [25]:
df2['emd_vector'] = df2['ex_sentence'].apply(get_sentence_vector)

<ipython-input-25-1eec6cecaf10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['emd_vector'] = df2['ex_sentence'].apply(get_sentence_vector)


In [26]:
df2.head()

,id,ex_sentence,emd_vector
0,1,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,"[[0.22070925, -0.08476977, 0.19659406, -0.1417..."
1,1,쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 ...,"[[0.18596092, -0.11348526, 0.104442544, -0.023..."
2,1,논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 ...,"[[0.13444728, -0.061891247, 0.18300448, -0.208..."
3,2,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,"[[0.15581228, -0.069558665, 0.14948563, -0.047..."
4,2,컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 따라 고소천사벽화마을과 자산마...,"[[0.24334654, -0.10914307, 0.21485253, -0.1025..."


In [28]:
df2['emd_vector'][0].shape

(1, 768)

In [29]:
df2['ex_sentence'][0]

'전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.'

# 데이터셋 나누기

In [34]:
len(df)

731931

In [40]:
# 태성 데이터
t_df = df[:365960]

In [38]:
# 수연 데이터
s_df = df[365960:]

# 벡터화

In [ ]:
def get_sentence_vector(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    input_ids = tf.constant([input_ids])
    outputs = model(input_ids)
    sentence_vector = outputs.pooler_output
    return sentence_vector.numpy()

In [1]:
# 태성
tqdm.pandas()

t_df['emd_vector'] = t_df['ex_sentence'].progress_apply(get_sentence_vector)
df.to_csv('/content/drive/MyDrive/god/data/tae_emdvector_table.csv',index=False)
print('데이터 저장 완료!')

In [ ]:
# 수연
tqdm.pandas()

s_df['emd_vector'] = s_df['ex_sentence'].progress_apply(get_sentence_vector)
df.to_csv('/content/drive/MyDrive/god/data/sue_emdvector_table.csv',index=False)
print('데이터 저장 완료!')